# THn visualization with RootInteractive
### Import libraries

* colorcet optional (for more ROOT colz like color pallete import it and use cc.b_linear_kbgoy_20_95_c57)

In [1]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
import base64
import ctypes
import logging
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
import colorcet as cc

Welcome to JupyROOT 6.24/06
x bokehVisJS3DGraph.ts
x HistogramCDS.ts
Import  CDSCompress.ts
Import  DownsamplerCDS.ts
Import  CDSAlias.ts
Import  CDSJoin.ts


### Load the THnSparse file

In [4]:
myFile = ROOT.TFile.Open("~/swalice/TPCQCVis/data/examples/THn/hNdPID.root")
hist = myFile.ExpertVis
thn = hist.Get("ExpertVis;1")
dim_count = thn.GetNdimensions()
dimensions = []
bin_count = []
bin_min = []
bin_max = []

for i in range(dim_count):
    dimensions.append(thn.GetAxis(i).GetTitle())
    bin_count.append(thn.GetAxis(i).GetNbins())
    bin_min.append(thn.GetAxis(i).GetXmin())
    bin_max.append(thn.GetAxis(i).GetXmax())
    
bin_centers = np.empty((dim_count, max(bin_count)))
for i in range(dim_count):
    for j in range(bin_count[i]):
        bin_centers[i][j] = thn.GetAxis(i).GetBinCenter(j+1)
        
thn.ls()


OBJ: THnSparseT<TArrayF>	hNdPid	Sprase Nd ExpertVis histogram for PID variables : 0 at: 0x16a933750


### Convert THnSparse to pandas.DataFrame
* the binning parameters should be known (ToDo: investigate if this can be extracted from the THn) 

In [5]:
bin_width = [(bin_max[x]-bin_min[x])/(bin_count[x]+1) for x in range(dim_count)]

tracks = [];
coord = (ctypes.c_int*dim_count)()
#for Bin in range(thn.GetNbins()):
for Bin in range(100000):
    val = thn.GetBinContent(Bin,coord)
    tracks.append(val)
    for Dim in range(dim_count):
        #tracks.append(bin_min[Dim]+(coord[Dim]-0.5)*bin_width[Dim])
        tracks.append(bin_centers[Dim][coord[Dim]-1])
    
df = pd.DataFrame(np.reshape(tracks, newshape=(int(len(tracks)/(dim_count+1)), dim_count+1)), 
                         columns = ['Weight','P','dEdxTot','dEdxMax','Phi','Tgl','Snp','NClusters','Sign'])
initMetadata(df)

df.head()

,Weight,P,dEdxTot,dEdxMax,Phi,Tgl,Snp,NClusters,Sign
0,19.0,0.280724,52.661151,16.936209,3.98125,-0.15,0.25,130.0,1.0
1,9.0,0.930446,48.027541,18.829414,0.65625,-0.35,0.15,134.0,1.0
2,2.0,1.080793,57.741804,20.934250,5.11875,-0.15,0.15,134.0,-1.0
3,36.0,1.255435,63.312629,20.934250,1.96875,-1.05,0.05,138.0,-1.0
4,8.0,0.280724,57.741804,20.934250,6.08125,0.15,-0.35,138.0,-1.0


## Plot as scatter plot
Doesn't make too much sense for us here since the data from all the dimensions are not aggregated

In [6]:
output_file("thn_scatter.html")

figureArray = [
    [['P'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['NClusters'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Phi'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Tgl'], ['dEdxTot'], {"colorZvar": "Weight"}],
    {"size":2, "legend_options": {"label_text_font_size": "13px"}}
]
layout = {
    "dEdx (a.u.) vs p (GeV/c)": [
        [0, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs ncls": [
        [1, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs φ (rad)": [
        [2, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs tanλ": [
        [3, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]]
}
tooltips = []
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'sizing_mode': 'scale_width'}]
    }    
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=layout,
                      tooltips=tooltips, widgetLayout=widgetLayoutDesc)

[['P'], ['dEdxTot'], {'colorZvar': 'Weight'}]
test
[['NClusters'], ['dEdxTot'], {'colorZvar': 'Weight'}]
test
[['Phi'], ['dEdxTot'], {'colorZvar': 'Weight'}]
test
[['Tgl'], ['dEdxTot'], {'colorZvar': 'Weight'}]
test


## As 1D histograms
Makes sense

In [89]:
output_file("thn_1D.html")

histo1D = lambda x: {"name": x + "_hist",
                     "variables": [x],
                     "nbins": bin_count[dimensions.index(x)],
                     "range": [bin_min[dimensions.index(x)],bin_max[dimensions.index(x)]],
                     "weights": "Weight"}

histoArray = [
    histo1D("dEdxTot"),
    histo1D("NClusters"),
    histo1D("Phi"),
    histo1D("Tgl"),
    histo1D("P"),
    histo1D("dEdxMax"),
    histo1D("Sign")
]
figureArray = [
    [['dEdxTot'],['dEdxTot_hist'],{'size': 5}],
    [['NClusters'],['NClusters_hist'],{'size': 5}],
    [['Phi'],['Phi_hist'],{'size': 5}],
    [['Tgl'],['Tgl_hist'],{'size': 5}],
    [['P'],['P_hist'],{'size': 5}],
    [['dEdxMax'],['dEdxMax_hist'],{'size': 5}],
    [['Sign'],['Sign_hist'],{'size': 5}]
]
figureLayoutDesc=[
    [[0, 1], [2,3], [4,6], {'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width'}
]
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'width':1}]
    }
tooltips = []
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, plot_width= 500, histogramArray=histoArray)

## As 2D histograms
Also makes sense

In [92]:
output_file("thn_2D.html")

histo2D = lambda x,y : {"name": y + "_vs_" + x,
                        "variables": [x, y],
                        "nbins": [bin_count[dimensions.index(x)], bin_count[dimensions.index(y)]],
                        "range": [[bin_min[dimensions.index(x)],bin_max[dimensions.index(x)]],[bin_min[dimensions.index(y)],bin_max[dimensions.index(y)]]],
                        "weights": "Weight"}

histoArray = [
    histo2D("P","dEdxMax"),
    histo2D("NClusters", "dEdxTot"),
    histo2D("Phi", "dEdxTot"),
    histo2D("Tgl", "dEdxTot")
]
figureArray = [
    [['P'], ['dEdxMax_vs_P']],
    [['NClusters'], ['dEdxTot_vs_NClusters']],
    [['Phi'], ['dEdxTot_vs_Phi']],
    [['Tgl'], ['dEdxTot_vs_Tgl']],
    ["tableHisto", {"rowwise": True}]
]
figureLayoutDesc={
    "dEdx_vs_p" : [[0,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_ncls" : [[1,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_phi" : [[2,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_tgl" : [[3,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}]
}
tooltips = []
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'width':1}]
    }
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, histogramArray=histoArray, plot_width= 500,
                            colors="Bokeh",rescaleColorMapper=True, nCols = 4)